In [ ]:
import polars as pl

In [ ]:
POPULATION_FILEPATH = '../../data/01_raw/popolazione_2023.csv'

In [ ]:
df = (
    pl.read_csv(POPULATION_FILEPATH, separator=';')
    .filter(
        (pl.col('FREQ') == 'A') &
        (pl.col('INDICATOR') == 'RESPOP_AV') &
        (pl.col('AGE_NOCLASS') == 'TOTAL') &
        (pl.col('TIME_PERIOD') == 2023)
    )    .with_columns(
        pl.col("Osservazione").cast(pl.Float64)  # Ensure numeric
    )
)

agg_df = (
    df.group_by(["REF_AREA", "Territorio",'TIME_PERIOD'])
    .agg([
        pl.when((pl.col("GENDER") == "F") & (pl.col("CITIZENSHIP") == "ITL"))
          .then(pl.col("Osservazione"))
          .otherwise(0.0)
          .sum()
          .alias("total_f_itl"),

        pl.when((pl.col("GENDER") == "M") & (pl.col("CITIZENSHIP") == "ITL"))
          .then(pl.col("Osservazione"))
          .otherwise(0.0)
          .sum()
          .alias("total_m_itl"),

        pl.when((pl.col("GENDER") == "F") & (pl.col("CITIZENSHIP") == "FRGAPO"))
          .then(pl.col("Osservazione"))
          .otherwise(0.0)
          .sum()
          .alias("total_f_frgapo"),

        pl.when(pl.col("GENDER") == "M")
          .then(pl.col("Osservazione"))
          .otherwise(0.0)
          .sum()
          .alias("total_m"),

        pl.when(pl.col("GENDER") == "F")
          .then(pl.col("Osservazione"))
          .otherwise(0.0)
          .sum()
          .alias("total_f"),

        pl.col("Osservazione").sum().alias("total_all")
    ])
)
agg_df

In [ ]:
OUTPUT_FILEPATH = '../../data/02_primary/italy_population.parquet'
agg_df.write_parquet(OUTPUT_FILEPATH)